In [159]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier as KNN

In [160]:
full_data = np.genfromtxt('verified_pb.data',
                     dtype=None,
                     delimiter='\t',
                     encoding=None)
print(len(full_data))
full_data[:10]

1520


array([(1, 1, 1, 'IY', 160., 240., 2280., 2850.),
       (1, 1, 1, 'IY', 186., 280., 2400., 2790.),
       (1, 1, 2, 'IH', 203., 390., 2030., 2640.),
       (1, 1, 2, 'IH', 192., 310., 1980., 2550.),
       (1, 1, 3, 'EH', 161., 490., 1870., 2420.),
       (1, 1, 3, '*EH', 155., 570., 1700., 2600.),
       (1, 1, 4, '*AE', 140., 560., 1820., 2660.),
       (1, 1, 4, 'AE', 180., 630., 1700., 2550.),
       (1, 1, 5, 'AH', 144., 590., 1250., 2620.),
       (1, 1, 5, 'AH', 148., 620., 1300., 2530.)],
      dtype=[('f0', '<i8'), ('f1', '<i8'), ('f2', '<i8'), ('f3', '<U3'), ('f4', '<f8'), ('f5', '<f8'), ('f6', '<f8'), ('f7', '<f8')])

In [161]:
x = np.array([[s[4], s[5], s[6], s[7]] for s in full_data], dtype=("float"))
y = np.array([s[2] for s in full_data], dtype=("int"))

x_train, x_0, y_train, y_0 = train_test_split(x, y, test_size=0.2, random_state=42)
x_valid, x_test, y_valid, y_test = train_test_split(x_0, y_0, test_size=0.5, random_state=42)

In [162]:
class Baseline():
    def __init__(self, nc):
        self.nc = nc

    def train(self, x, y):
        centers = []
        for i in range(1, self.nc+1):
            pos = np.where(i == y)
            center = sum(x[pos])/len(pos[0])
            centers.append(center)
        self.centers = np.array(centers)
    
    def predict(self, x):
        dist_matrix = np.zeros((len(x), self.nc))
        for i in range(self.nc):
            z = np.array([self.centers[i] for _ in range(len(x))])
            dist = np.sum((x-z)**2, axis=1)
            dist_matrix[:, i] = dist
        
        y = np.argmin(dist_matrix, axis=1)+1
        return y

In [163]:
classifier = Baseline(10)
classifier.train(x_train, y_train)
pred = classifier.predict(x_0)
sum((pred == y_0).astype("float"))/len(y_0)

0.5197368421052632

A modelleket csak az F értékeken tanítjuk (a nemet, valamint, hogy hányadik alany, figyelmen kívül hagyjuk). Baseline modellként minden osztályra kiszámoljuk az osztály "középpontját", és azt az osztályt predikáljuk, amelynek a középpontjához a legközelebb van az adatpont (Euklideszi távolsággal számolunk). Ez a validációs és teszt adatokon ~52% accuracyt ad.

In [164]:
for k in range(1, 10, 2):
    model = KNN(n_neighbors = k)
    model.fit(x_train, y_train)
    pred = model.predict(x_valid)
    accuracy = sum((pred == y_valid).astype("float"))/len(y_valid)
    print(f"{k} szomszéddal a validációs accuracy score: {accuracy}")

1 szomszéddal a validációs accuracy score: 0.8486842105263158
3 szomszéddal a validációs accuracy score: 0.8421052631578947
5 szomszéddal a validációs accuracy score: 0.868421052631579
7 szomszéddal a validációs accuracy score: 0.868421052631579
9 szomszéddal a validációs accuracy score: 0.8486842105263158


A legjobb validációs accuracy scoret 5 és 7 szomszéd esetén kapjuk, ezeket megvizsgáljuk a tesztadaton is.

In [165]:
model_5 = KNN(n_neighbors = 5)
model_5.fit(x_train, y_train)
pred = model_5.predict(x_test)
accuracy = sum((pred == y_test).astype("float"))/len(y_test)
print(f"{5} szomszéddal a test accuracy score: {accuracy}")
model_7 = KNN(n_neighbors = 7)
model_7.fit(x_train, y_train)
pred = model_7.predict(x_test)
accuracy = sum((pred == y_test).astype("float"))/len(y_test)
print(f"{7} szomszéddal a test accuracy score: {accuracy}")

5 szomszéddal a test accuracy score: 0.8157894736842105
7 szomszéddal a test accuracy score: 0.8486842105263158


Hozzátanítva a modellekhez a validációs adatot is:

In [166]:
x_valtrain = np.concatenate((x_train, x_valid), axis=0)
y_valtrain = np.concatenate((y_train, y_valid), axis=0)


model_5 = KNN(n_neighbors = 5)
model_5.fit(x_valtrain, y_valtrain)
pred = model_5.predict(x_test)
accuracy = sum((pred == y_test).astype("float"))/len(y_test)
print(f"{5} szomszéddal a test accuracy score: {accuracy}")
model_7 = KNN(n_neighbors = 7)
model_7.fit(x_valtrain, y_valtrain)
pred = model_7.predict(x_test)
accuracy = sum((pred == y_test).astype("float"))/len(y_test)
print(f"{7} szomszéddal a test accuracy score: {accuracy}")

5 szomszéddal a test accuracy score: 0.8092105263157895
7 szomszéddal a test accuracy score: 0.8223684210526315


A továbbiakban a KNN algoritmust próbáltuk ki a feladatra, 1, 3, 5, 7 és 9 legközelebbi szomszéd figyelembevételével. A validáció során azt kaptuk, hogy 5 és 7 szomszéd esetén a legjobb a modell teljesítménye, így ezeket teszteltük, a tesztek során a k=7 szomszéd bizonyult a legjobbnak, ezzel a modell 82-84% accuracyt ért el a tesztadaton.